# Notebook E-tivity 3 CE4021 Task 2

Student name: Bartlomiej Mlynarkiewicz

Student ID: 17241782

<hr style=\"border:2px solid gray\"> </hr>

## Imports

If you believe required imports are missing, please contact your moderator.

<hr style=\"border:2px solid gray\"> </hr>

## Task 2

Using Etivity3-Task2.ipynb from the Gitlab repository and the dataset contained therein, create a Naive Bayes Classifier to filter incoming mail for SPAM.

The notebook provides a small dataset of previous 'emails' (please note the absence of punctuation which simplifies the coding challenge somewhat). Previous wanted emails are contained in previous_ham. Previous unwanted emails are contained in previous_spam. 

Write code using Bayes' Rule to determine whether the messages contained in new_emails are HAM or SPAM. Compare the decisions your classifier takes with the label associated with the messages (indicated by the key under which they are stored in the new_emails dictionary. 

If time permits, add the code required to allow your classifier to learn from the email messages contained in new_emails. Note that this functionality is required to be graded in the Exemplary category. 

HINTS:

1. Use functions to divide up the task in smaller components. It is useful to work through the problem by hand to get a handle on what functions would be useful.
2. Choose a suitable threshold of 'spamicity' (or 'spaminess') to distinguish between spam and ham messages in this dataset. 

Use the below information to create a Naive Bayes SPAM filter. Test your filter using the messages in new_emails. You may add as many cells as you require to complete the task.

### Background

A Naive Bayes classifier is a probabilistic machine learning model that’s used for classification task. The crux of the classifier is based on the Bayes theorem.


$$
P(A|B) = \frac{P(A|B)P(A)}{P(B)}
$$

Using Bayes theorem, we can find the probability of A happening, given that B has occurred. Here, B is the evidence and A is the hypothesis. The assumption made here is that the features are independent. That is presence of one particular feature does not affect the other. Hence it is called naive.

### Data

In [1]:
previous_spam = ['send us your password', 'review our website', 'send your password', 'send us your account']
previous_ham = ['Your activity report','benefits physical activity', 'the importance vows']
new_emails = {'spam':['renew your password', 'renew your vows'], 'ham':['benefits of our account', 'the importance of physical activity']}

### Laplace Smoothing

Laplace smoothing is a smoothing technique that handles the problem of zero probability in Naïve Bayes.

$$
P(w_i|Spam) = \frac{N_{wi|Spam} + \alpha}{N_{Spam} + \alpha * N_{UniqueWords}}
$$

- $\alpha$ represents the smoothing parameter.
- $N_{wi|Spam}$ represents the number of times the word occurs in SPAM emails.
- $N_{Spam}$represents the count of words in SPAM emails.
- $N_{UniqueWords}$ represents the count of unique words in both SPAM and HAM previous emails.

$$
P(w_i|Ham) = \frac{N_{wi|Ham} + \alpha}{N_{Ham} + \alpha * N_{UniqueWords}}
$$

- $\alpha$ represents the smoothing parameter.
- $N_{wi|Ham}$ represents the number of times the word occurs in HAM emails.
- $N_{Ham}$ represents the count of words in HAM emails.
- $N_{UniqueWords}$ represents the count of unique words in both SPAM and HAM previous emails.


If a value of $\alpha$ != 0 is choosen, the probability will no longer be zero even if a word is not present in the training dataset.

In the below example, I was planning to use the approach of comparing the $P(Spam|wi)$ is grater than the $P(Ham|wi)$ then the below implementation classified the email as SPAM. However based on the requiremtns I choose a threeshold of one as 

### Compare the predicted values with the actual values to measure how good our spam filter is with classifying new emails. 

$$
\text{Accuracy} = \frac{\text{no. of correctly classified emails}}{\text{total number of classified emails}}
$$

### Implementation

Below is a `defaultdict` implementation, similar to `defaultdict` provided by the `collections` library. The standard dictionary includes the method setdefault() for retrieving a value and establishing a default if the value does not exist. By contrast, defaultdict lets the caller specify the default(value to be returned) up front when the container is initialized. This is to avoid `KeyError` being thrown if a key doesn't exist within the `dict`.

In [2]:
def defaultdict(default_type) -> 'DefaultDict':
    """
    Returns a defaultdict object.
    
    Parameters
    ----------
    default_type
        A default type which is used to set the value of item.
            
    Returns
    -------
    DefaultDict
        Returns a an instance of DefaultDict.
    """
    class DefaultDict(dict):
        def __getitem__(self, key):
            if key not in self:
                dict.__setitem__(self, key, default_type())
            return dict.__getitem__(self, key)
    return DefaultDict()

In [46]:
class SpamFilter():
    def __init__(self):
        self.spam_word_counts = defaultdict(int)
        self.ham_word_counts = defaultdict(int)
        self.total_spam_messages = 0
        self.total_ham_messages = 0
        self.total_words_in_spam = 0
        self.total_words_in_ham = 0
        
    def __remove_fill_words(self, text):
        """
        Removes fill words from a given string.

        Parameters
        ----------
        text: str
            A string to remove fill words from.

        Returns
        -------
        list[str]
            Returns a list of strings (tokens).
        """
        
        fill_words = [
            "as",
            "to",
            "and",
            "the",
            "a",
            "of",
            "our"
        ]

        return [word for word in text.split() if word not in fill_words]
    
    def __remove_punctuation_and_fill_words(self, text: str) -> list[str]:
        """
        Removes punctuations from the passed in text.

        Parameters
        ----------
        text: str
            A string

        Returns
        -------
        str
            Returns a string without punctuations.
        """
        # String of ASCII characters which are considered punctuation characters in the C locale.
        # Copied from strings.punctuation documentation
        punctuations = '''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~'''
        clean_text = ''.join(char for char in text if char not in punctuations).lower()
        return self.__remove_fill_words(clean_text)
    
    def pritty_print(self, lst, msg):
        """
        Removes punctuations from the passed in text.

        Parameters
        ----------
        text: str
            A string

        Returns
        -------
        str
            Returns a string without punctuations.
        """
        print(f"\033[1m{msg}\033[0m\n")
        for index, obj in enumerate(lst):
            print(f"{index+1}.\033[1m{'Email:':<40}\033[0m {obj['email'] if 'email' in obj else obj}")
            if 'spamminess_score' in obj:
                print(f"{index+1}.\033[1m{'Spaminess Score:':<40}\033[0m {obj['spamminess_score']}\n")
        print()
        
    def accuracy(self, classified_emails: dict, new_emails: dict):
        """
        Calculates and prints the accuracy of the predictions.

        Parameters
        ----------
        classified_emails: DefaultDict
            Dict of classified emails
            
        new_emails: DefaultDict
            Dict of emails to compare the classified emails to

        Returns
        -------
        None
        """
        total = sum(len(new_emails[key]) for key in list(new_emails.keys()))

        # Get the common elements in both sets
        spam_count = len(list(set(new_emails['spam']).intersection(set(email['email'] for email in classified_emails['spam']))))
        ham_count = len(list(set(new_emails['ham']).intersection(set(email['email'] for email in classified_emails['ham']))))

        print(f"\033[1m{'Accuracy:':<40}\033[0m\n")
        print(f"\033[1m{'Correct:':<40}\033[0m{spam_count + ham_count}")
        print(f"\033[1m{'Incorrect:':<40}\033[0m{total - (spam_count+ham_count)}")
        print(f"\033[1m{'Accuracy:':<40}\033[0m{round(((spam_count+ham_count)/total)*100)}%")
    
    def train_naive_bayes(self, previous_ham: list[str], previous_spam: list[str]):
        """
        Initializes the required constants required for classification. 

        Parameters
        ----------
        previous_ham: list[str]
            A list of previous emails that were catagorised as HAM

        previous_spam: list[str]
            A list of previous emails that were catagorised as SPAM

        Returns
        -------
        spam_word_counts: DefaultDict
            Dictionary with frequency of each word in previous spam emails - - Nwi|spam

        ham_word_counts: DefaultDict
            Dictionary with frequency of each word in previous ham emails - Nwi|ham

        total_spam_messages: int
            The count of previous spam emails

        total_ham_messages: int
            The count of previous ham emails

        total_words_in_spam: int 
            The count of words in previous spam emails - Nspam

        total_words_in_ham: int
            The count of words in previous ham emails - Nham
        """
        # Count the number of previous spam and ham email
        self.total_spam_messages = len(previous_spam)
        self.total_ham_messages = len(previous_ham)

        # Count the number of words in previous spam and ham emails
        self.total_words_in_spam = sum(len(email.split()) for email in previous_spam)
        self.total_words_in_ham = sum(len(email.split()) for email in previous_ham)

        # Count the frequency of each word in previous spam emails
        for email in previous_spam:
            words = self.__remove_punctuation_and_fill_words(email)
            for word in words:
                self.spam_word_counts[word] += 1

        # Count the frequency of each word in previous ham emails
        for email in previous_ham:
            words = self.__remove_punctuation_and_fill_words(email)
            for word in words:
                self.ham_word_counts[word] += 1
                
    def predict_naive_bayes(self, email: str):
        """
        Classified emails SPAM or HAM based on the comparsion between the ham and spam score. 

        Parameters
        ----------
        email: str
            A string to be used to classify.

        spam_word_counts: DefaultDict
            Dictionary with frequency of each word in previous spam emails - - Nwi|spam

        ham_word_counts: DefaultDict
            Dictionary with frequency of each word in previous ham emails - Nwi|ham

        total_spam_messages: int
            The count of previous spam emails

        total_ham_messages: int
            The count of previous ham emails

        total_words_in_spam: int 
            The count of words in previous spam emails - Nspam

        total_words_in_ham: int
            The count of words in previous ham emails - Nham

        Returns
        -------
        spamminess_score: float
            The ratio of the spam score to the ham score.

        prediction: str
            The email classification based on the ratio of the spam score to the ham score.
        """
        # Tokenize email and lower case each token
        email_tokenized = self.__remove_punctuation_and_fill_words(email)

        # Laplace smoothing
        alpha = 1

        # Calculate P(S) -> P(Spam)
        spam_score = self.total_spam_messages / (self.total_spam_messages + self.total_ham_messages)
        # Calculate P(¬S) -> P(Ham)
        ham_score = self.total_ham_messages / (self.total_spam_messages + self.total_ham_messages)

        # Count of unique words in both Spam and Ham
        no_of_uniqe_words = len(set().union(self.spam_word_counts.keys(), self.ham_word_counts.keys()))

        spam_score_tmp = []
        for word in email_tokenized:
            # Calculate the conditional probabilities P(w_i|Spam) and P(w_i|Ham)
            prob_word_given_spam = (self.spam_word_counts[word] + alpha) / (self.total_words_in_spam + (alpha*no_of_uniqe_words))
            prob_word_given_ham = (self.ham_word_counts[word] + alpha) / (self.total_words_in_ham + (alpha*no_of_uniqe_words))

            # Update the spam and ham scores by multiplying them with the calculated conditional probabilities for each word in the email.
            spam_score *= prob_word_given_spam
            ham_score *= prob_word_given_ham

        # The spamminess score is calculated as the ratio of the spam score to the ham score.
        #P(Spam|email)
        #P(Ham|email)
        spamminess_score = spam_score / ham_score
        # Classify SPAM if spam_score > ham_score
        prediction = 'spam' if spam_score > ham_score else 'ham' 

        return prediction, spamminess_score

    def classify(self, email_list: list[str]):
        classified_emails = {'spam': [], 'ham': []}

        for label, emails in email_list.items():
            for email in emails:
                prediction, spamminess_score = self.predict_naive_bayes(email)
                classified_emails[prediction].append({"email": email, "spamminess_score": spamminess_score})
        
        return classified_emails

### Run 1: Train Naive Bayes on the given set of previous emails

In [47]:
spam_filter = SpamFilter()
spam_filter.train_naive_bayes(previous_ham, previous_spam)
    
classified_emails = spam_filter.classify(new_emails)

spam_filter.pritty_print(previous_spam, "Previous SPAM")
spam_filter.pritty_print(previous_ham, "Previous HAM")

spam_filter.pritty_print(new_emails['spam'], "Actual SPAM")
spam_filter.pritty_print(new_emails['ham'], "Actual HAM")

spam_filter.pritty_print(classified_emails['spam'], "Bayes Classified SPAM")
spam_filter.pritty_print(classified_emails['ham'], "Bayes Classified HAM")

spam_filter.accuracy(classified_emails, new_emails)

Previous SPAM

1.Email:                                   send us your password
2.Email:                                   review our website
3.Email:                                   send your password
4.Email:                                   send us your account

Previous HAM

1.Email:                                   Your activity report
2.Email:                                   benefits physical activity
3.Email:                                   the importance vows

Actual SPAM

1.Email:                                   renew your password
2.Email:                                   renew your vows

Actual HAM

1.Email:                                   benefits of our account
2.Email:                                   the importance of physical activity

Bayes Classified SPAM

1.Email:                                   renew your password
1.Spaminess Score:                         4.327795559619975


Bayes Classified HAM

1.Email:                                   renew your

### Run 2: Train Naive Bayes on a larger training set

### Generated email subject lines to increase the training set to improve the models accuracy

In [5]:
extra_spam_samples = [
    "Exciting Product Launch: Don't Miss Our Latest Gadgets!",
    "Join Our Webinar: Digital Marketing Strategies for Success",
    "Adventure Retreats: Explore Remote Wilderness with Us",
    "Exclusive Sale: Get 40% Off on Latest Fashion Trends",
    "Mastering Time Management: Boost Your Productivity",
    "Culinary Journey: Explore International Cuisine with Us",
    "Career Advancement: Strategies for Success in the Workplace",
    "Sustainable Living: Eco-Friendly Tips for a Greener Future",
    "Art & Culture Festival: Celebrating Diversity in Our City",
    "Discover Hidden Treasures: Antique Collecting Insights",
    "Space Exploration News: Unprecedented Discoveries",
    "Smart Living: Transform Your Home with Technology",
    "Hiking Adventures: Conquer Majestic Peaks and Trails",
    "Women in Leadership: Empowering Female Entrepreneurs",
    "Historical Mysteries: Explore Ancient Civilizations",
    "eSports Phenomenon: Dive into Competitive Gaming",
    "Photography Mastery: Learn from the Pros",
    "Financial Planning: Tips for a Secure Future",
    "Mindfulness & Meditation: Find Inner Peace and Resilience",
    "Fashion Evolution: Styles through the Decades"
]

extra_ham_samples = [
    "Explore the Secrets of Ancient Egypt: Unearth Mysteries of the Pharaohs, Pyramids, and Hieroglyphs in Our Archaeological Expedition",
    "Revolutionize Your Home Office: Discover Ergonomic Furniture, High-Tech Gadgets, and Office Equipment for Ultimate Comfort and Productivity",
    "Journey to the Stars: Join Our Astronaut Training Program and Experience the Rigorous Preparations for Space Exploration",
    "Immerse Yourself in World Literature: Our Acclaimed Book Club Delves Deep into Timeless Classics and Engages in Thoughtful Literary Discussions",
    "Embark on an Epic Culinary Adventure: Experience the Art of Cooking with Michelin-Starred Chefs and Master the Secrets of Gourmet Cuisine",
    "Unleash Your Inner Explorer: Join Our Adventure Club and Traverse the Most Remote and Breathtaking Corners of the Globe",
    "Redefine Sustainable Living: Eco-Friendly Practices and Green Innovations for a More Environmentally Conscious Lifestyle",
    "Elevate Your Career: Uncover the Secrets of Effective Leadership and Team Management in Our Comprehensive Leadership Development Program",
    "Escape to Tropical Paradise: Our Luxury Island Retreat in the Maldives Offers Pristine Beaches, Sunshine, and Overwater Bungalows",
    "Transform Your Backyard into a Serene Oasis: Explore Our Luxurious Outdoor Furniture and Garden Decor Made from Sustainable Materials",
    "Dive Deep into Ocean Conservation: Join Our Marine Biology Expedition for Hands-On Research, Diving, and Wildlife Encounters",
    "Experience the Power of Giving: Our Philanthropy Symposium Focuses on Transforming Lives and Making a Positive Impact in Your Community",
    "A Journey Through Ancient History: Explore Mythology, Civilizations, and Archaeological Sites That Shaped Our World",
    "Discover the Future of Transportation: Get Behind the Wheel of Electric Vehicles with Advanced Autonomous Features and Eco-Friendly Mobility Solutions",
    "Charting New Horizons in Medicine: Explore the Latest Healthcare Innovations and Breakthroughs with Insights from Medical Professionals",
    "Unleash Your Creativity: Join Our Workshop Series with Renowned Artists and Innovators to Explore New Perspectives in Art and Design",
    "Elevate Your Financial Literacy: Enroll in Our Personal Finance Course Covering Budgeting, Investing, Retirement Planning, and Wealth Management",
    "Witness the Majesty of Opera: Our Exclusive Opera Tour Takes You to Prestigious Venues in Europe, Where You'll Meet Opera Stars and Enjoy World-Class Performances",
    "Savor the Flavors of the World: Join Our International Food Festival and Delight in Culinary Delights from Around the Globe",
    "The Future of Education: Explore Innovations in Learning and Teaching in Our Symposium with Expert Educators and Visionaries"
]

In [6]:
spam_filter = SpamFilter()
spam_filter.train_naive_bayes(previous_ham + extra_ham_samples, previous_spam + extra_spam_samples)
    
classified_emails = spam_filter.classify(new_emails)
    
spam_filter.pritty_print(previous_spam, "Previous SPAM")
spam_filter.pritty_print(previous_ham, "Previous HAM")

spam_filter.pritty_print(new_emails['spam'], "Actual SPAM")
spam_filter.pritty_print(new_emails['ham'], "Actual HAM")

spam_filter.pritty_print(classified_emails['spam'], "Bayes Classified SPAM")
spam_filter.pritty_print(classified_emails['ham'], "Bayes Classified HAM")

spam_filter.accuracy(classified_emails, new_emails)

1.008668242710796e-07
1.4194054926090199e-08
3.3399118796768296e-08
2.8262683922671978e-08
5.021000859114168e-06
2.129122188841289e-06
5.566519799461383e-09
1.884178928178132e-08
Previous SPAM

1.Email:                                   send us your password
2.Email:                                   review our website
3.Email:                                   send your password
4.Email:                                   send us your account

Previous HAM

1.Email:                                   Your activity report
2.Email:                                   benefits physical activity
3.Email:                                   the importance vows

Actual SPAM

1.Email:                                   renew your password
2.Email:                                   renew your vows

Actual HAM

1.Email:                                   benefits of our account
2.Email:                                   the importance of physical activity

Bayes Classified SPAM

1.Email:              

### Run 3: Add new emails to training set

In [7]:
spam_filter = SpamFilter()
spam_filter.train_naive_bayes(previous_ham + extra_ham_samples + new_emails['ham'], previous_spam + extra_spam_samples + new_emails['spam'])
    
classified_emails = spam_filter.classify(new_emails)
    
spam_filter.pritty_print(previous_spam, "Previous SPAM")
spam_filter.pritty_print(previous_ham, "Previous HAM")

spam_filter.pritty_print(new_emails['spam'], "Actual SPAM")
spam_filter.pritty_print(new_emails['ham'], "Actual HAM")

spam_filter.pritty_print(classified_emails['spam'], "Bayes Classified SPAM")
spam_filter.pritty_print(classified_emails['ham'], "Bayes Classified HAM")

spam_filter.accuracy(classified_emails, new_emails)

5.127737737312418e-07
1.3606365432631968e-08
2.563868868656209e-07
2.7212730865263935e-08
4.882033637399531e-06
6.23171536814544e-06
5.341393476367102e-09
5.442546173052786e-08
Previous SPAM

1.Email:                                   send us your password
2.Email:                                   review our website
3.Email:                                   send your password
4.Email:                                   send us your account

Previous HAM

1.Email:                                   Your activity report
2.Email:                                   benefits physical activity
3.Email:                                   the importance vows

Actual SPAM

1.Email:                                   renew your password
2.Email:                                   renew your vows

Actual HAM

1.Email:                                   benefits of our account
2.Email:                                   the importance of physical activity

Bayes Classified SPAM

1.Email:                

<hr style=\"border:2px solid gray\"> </hr>

## Reflection

Having a small train set I was still able to achieve a quite high accuracy result at 75%, increasing the train set didn't increase the accuracy of the predictions. Including the new emails list in run `three` resulted in a 100% accuracy as the has seen all of the token during the training stage that the new emails were being classified and therefore was able to correctly classify the new email set.

It would be interesting to train the model on a substantial sized data set reflected on real list of SPAM emails.